In [1]:
import time, math, threading, asyncio, random, concurrent.futures
from datetime import datetime

try:
    import aiohttp
    AIOHTTP_OK = True
except Exception:
    AIOHTTP_OK = False

print("Imports OK | aiohttp:", "OK" if AIOHTTP_OK else "NOT INSTALLED (поставь: %pip install aiohttp)")

Imports OK | aiohttp: OK


In [6]:
print("\n==================== ЗАДАЧА 1 ====================")

def sync_calculate(operation, a, b, delay):
    print(f"Начало операции {a} {operation} {b}")
    time.sleep(delay)

    if operation == '+':
        result = a + b
    elif operation == '-':
        result = a - b
    elif operation == '*':
        result = a * b
    elif operation == '/':
        result = a / b if b != 0 else 'Ошибка: деление на ноль'
    else:
        result = 'Неизвестная операция'

    print(f"Конец операции {a} {operation} {b} = {result}")
    return result

start_time = time.time()
results_1 = []
results_1.append(sync_calculate('+', 15, 25, 2))
results_1.append(sync_calculate('-', 40, 18, 1))
results_1.append(sync_calculate('*', 12, 8, 3))
results_1.append(sync_calculate('/', 100, 5, 1))
end_time = time.time()

print(f"Общее время выполнения: {end_time - start_time:.2f} секунд")
print(f"Результаты: {results_1}")



print("\n==================== ЗАДАЧА 2 ====================")

def download_file(filename, size):
    download_time = size * 0.1
    print(f"Начало загрузки {filename} ({size} МБ)")

    for i in range(5):
        time.sleep(download_time / 5)
        progress = (i + 1) * 20
        print(f"{filename}: {progress}% загружено")

    print(f"Завершена загрузка {filename}")

files = [
    ("document.pdf", 10),
    ("image.jpg", 5),
    ("video.mp4", 20),
    ("archive.zip", 15)
]

start_time = time.time()
threads = []

for filename, size in files:
    t = threading.Thread(target=download_file, args=(filename, size))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

end_time = time.time()
print(f"Общее время загрузки: {end_time - start_time:.2f} секунд")



print("\n==================== ЗАДАЧА 3 ====================")

import time
import math
import concurrent.futures

def factorial_digits(n: int):
    s = 0.0
    for k in range(1, n + 1):
        s += math.log10(k)
    digits = int(s) + 1 if n > 1 else 1
    return ("factorial_digits", n, digits)

def prime_check(n: int):
    if n < 2:
        return ("prime", n, False)
    for i in range(2, int(math.isqrt(n)) + 1):
        if n % i == 0:
            return ("prime", n, False)
    return ("prime", n, True)

jobs = [
    (factorial_digits, 10000),
    (factorial_digits, 8000),
    (prime_check, 10000019),
    (prime_check, 10000033),
]

print("=== ПАРАЛЛЕЛЬНО (ThreadPoolExecutor) ===")
t0 = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as ex:
    futures = [ex.submit(func, arg) for func, arg in jobs]
    par_results = [f.result() for f in futures]
par_time = time.time() - t0

for kind, arg, res in par_results:
    if kind == "factorial_digits":
        print(f"digits({arg}!) -> {res}")
    else:
        print(f"prime({arg}) -> {res}")

print("\n=== ПОСЛЕДОВАТЕЛЬНО (sync) ===")
t0 = time.time()
sync_results = [func(arg) for func, arg in jobs]
sync_time = time.time() - t0

for kind, arg, res in sync_results:
    if kind == "factorial_digits":
        print(f"digits({arg}!) -> {res}")
    else:
        print(f"prime({arg}) -> {res}")

print("\nСравнение времени:")
print(f"ThreadPool: {par_time:.2f} сек")
print(f"Sync:       {sync_time:.2f} сек")
if par_time > 0:
    print(f"Ускорение:  {sync_time/par_time:.2f}x")



print("\n==================== ЗАДАЧА 4 ====================")

async def fetch_url(session, url, name):
    print(f"Начало загрузки {name}")
    try:
        async with session.get(url) as response:
            content = await response.text()
            print(f"Завершена загрузка {name}, статус: {response.status}")
            return (name, len(content))
    except Exception as e:
        print(f"Ошибка при загрузке {name}: {e}")
        return (name, 0)

async def run_task4():
    if not AIOHTTP_OK:
        print("Нет aiohttp")
        return []

    urls = [
        ("https://httpbin.org/delay/1", "Сайт 1"),
        ("https://httpbin.org/delay/2", "Сайт 2"),
        ("https://httpbin.org/delay/1", "Сайт 3"),
        ("https://httpbin.org/delay/3", "Сайт 4"),
    ]

    start_time = time.time()
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_url(session, url, name) for url, name in urls]
        results = await asyncio.gather(*tasks)
    end_time = time.time()

    print(f"Общее время выполнения: {end_time - start_time:.2f} секунд")
    print("Размеры контента:", results)
    return results

try:
    _t4 = await run_task4()
except Exception as e:
    print("ЗАДАЧА 4 ошибка:", e)



print("\n==================== ЗАДАЧА 5 ====================")

def io_task(name, duration):
    time.sleep(duration)
    return f"{name} completed"

async def async_io_task(name, duration):
    await asyncio.sleep(duration)
    return f"{name} completed"

tasks = [("Task1", 2), ("Task2", 3), ("Task3", 1), ("Task4", 2), ("Task5", 1)]

t0 = time.time()
sync_out = [io_task(n, d) for n, d in tasks]
sync_t = time.time() - t0

t0 = time.time()
thread_out = [None] * len(tasks)

def worker(i, name, dur):
    thread_out[i] = io_task(name, dur)

ths = []
for i, (n, d) in enumerate(tasks):
    th = threading.Thread(target=worker, args=(i, n, d))
    th.start()
    ths.append(th)
for th in ths:
    th.join()
thread_t = time.time() - t0

t0 = time.time()
try:
    with concurrent.futures.ProcessPoolExecutor(max_workers=5) as ex:
        proc_out = list(ex.map(lambda x: io_task(*x), tasks))
    proc_t = time.time() - t0
except Exception as e:
    proc_out = []
    proc_t = None

async def run_async_5():
    return await asyncio.gather(*[async_io_task(n, d) for n, d in tasks])

t0 = time.time()
async_out = await run_async_5()
async_t = time.time() - t0

print("\n=== АНАЛИЗ РЕЗУЛЬТАТОВ ===")
print(f"Синхронное время: {sync_t:.2f} сек")
print(f"Многопоточное время: {thread_t:.2f} сек")
if proc_t is not None:
    print(f"Многопроцессное время: {proc_t:.2f} сек")
else:
    print("Многопроцессное время: (нет)")
print(f"Асинхронное время: {async_t:.2f} сек")



print("\n==================== 6 (ДОП) ====================")

async def scheduled_task(name, priority, duration):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] '{name}' (prio {priority}) start")
    await asyncio.sleep(duration)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] '{name}' end")
    return name

tasks_with_priority = [
    ("Экстренная задача", 1, 1),
    ("Важная задача", 2, 2),
    ("Обычная задача A", 3, 3),
    ("Обычная задача B", 3, 2),
    ("Фоновая задача", 4, 5),
]

tasks_sorted = sorted(tasks_with_priority, key=lambda x: x[1])
sem = asyncio.Semaphore(2)

async def run_one(name, pr, dur):
    async with sem:
        return await scheduled_task(name, pr, dur)

dop6_results = await asyncio.gather(*[run_one(n, p, d) for n, p, d in tasks_sorted])
print("Результаты:", dop6_results)



print("\n==================== 7 (ДОП) ====================")

def process_data(item):
    process_time = random.uniform(0.5, 2.0)
    time.sleep(process_time)
    result = item * 2
    print(f"Обработан {item} -> {result} ({process_time:.2f}с)")
    return result

data = list(range(1, 11))

def run_pool(workers: int):
    t0 = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as ex:
        results = list(ex.map(process_data, data))
    return time.time() - t0, results

t2, _ = run_pool(2)
t4, _ = run_pool(4)
t8, _ = run_pool(8)

print("\n=== СРАВНЕНИЕ ПРОИЗВОДИТЕЛЬНОСТИ ===")
print(f"Пул 2 потоков: {t2:.2f} сек")
print(f"Пул 4 потоков: {t4:.2f} сек")
print(f"Пул 8 потоков: {t8:.2f} сек")



print("End")


==================== ЗАДАЧА 1 ====================
Начало операции 15 + 25
Конец операции 15 + 25 = 40
Начало операции 40 - 18
Конец операции 40 - 18 = 22
Начало операции 12 * 8
Конец операции 12 * 8 = 96
Начало операции 100 / 5
Конец операции 100 / 5 = 20.0
Общее время выполнения: 7.02 секунд
Результаты: [40, 22, 96, 20.0]

==================== ЗАДАЧА 2 ====================
Начало загрузки document.pdf (10 МБ)
Начало загрузки image.jpg (5 МБ)
Начало загрузки video.mp4 (20 МБ)
Начало загрузки archive.zip (15 МБ)
image.jpg: 20% загружено
document.pdf: 20% загружено
image.jpg: 40% загружено
archive.zip: 20% загружено
image.jpg: 60% загружено
document.pdf: 40% загруженоvideo.mp4: 20% загружено

image.jpg: 80% загружено
image.jpg: 100% загружено
Завершена загрузка image.jpg
document.pdf: 60% загруженоarchive.zip: 40% загружено

video.mp4: 40% загружено
document.pdf: 80% загружено
archive.zip: 60% загружено
document.pdf: 100% загружено
Завершена загрузка document.pdf
archive.zip: 80% загру